This is playing with pytorch framework for EHR modeling. In general, a patient's health record can be represented as a sequence of visits. Each visit has certain features, and can be represented as a list of medical codes.

For simplicity, we are starting with the data structure that a patient's health record is a list of list, following the line of work from Jimeng Sun's lab. We will use codes from Ed Choi to manipulate the data. 

The core model is an RNN , either LSTM, GRU or Vanilla RNN.

# todos:
* None for now

In [1]:
%matplotlib inline
from __future__ import print_function, division
from io import open
import string
import re
import random
import sklearn 
from sklearn.metrics import roc_auc_score
import plotly.plotly as py 
import plotly.graph_objs as go
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.utils.data as Data

use_cuda = torch.cuda.is_available()
torch.cuda.set_device(1)

import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle

from torchviz import make_dot, make_dot_from_trace

# for windows only    
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
#os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
#os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [2]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=1
use_cuda

True

In [3]:
merged_set= pickle.load(open('/data/projects/py_ehr_2/Data/Readm_h_143cl2_combined_tp.all', 'rb'), encoding='bytes')

dataSize = len(merged_set)
nTest = int(0.2 * dataSize)
nValid = int(0.1 * dataSize)
    
random.shuffle(merged_set)

test_sl = merged_set[:nTest]
valid_sl = merged_set[nTest:nTest+nValid]
train_sl = merged_set[nTest+nValid:]

print (len(train_sl),len(valid_sl),len(test_sl))

12629 1803 3607


In [4]:
x=train_sl[2]
print(x[-1])
print(x[-1][-1][1:])

[[[0], [1, 55, 69, 56, 19, 57, 429, 7, 5, 61, 62, 154, 63, 13, 36, 11]], [[21], [61, 56, 4, 57, 60, 429, 7, 55, 36, 62, 754, 63, 13, 11, 69, 1]], [[1], [61, 56, 57, 60, 429, 7, 55, 4, 62, 63, 13, 36, 11, 1, 69, 754]], [[29], [61, 56, 4, 57, 5, 429, 7, 55, 62, 1, 68, 63, 1407, 13, 36, 11, 69, 65]], [[42], [60, 7, 429, 5, 61, 57, 4, 56, 55, 1, 62, 536, 754, 68, 63, 13, 36, 11, 69]], [[14], [61, 56, 4, 57, 143, 429, 7, 55, 62, 140, 63, 13, 36, 11, 69, 1]], [[5], [61, 4, 57, 5, 429, 7, 55, 56, 62, 63, 13, 36, 11, 69, 1, 754]], [[7], [61, 56, 4, 57, 5, 429, 7, 55, 62, 68, 63, 13, 36, 11, 69, 1]], [[17], [55, 56, 4, 57, 5, 429, 7, 61, 1, 68, 63, 13, 36, 11, 69, 62]], [[4], [62, 4, 5, 429, 7, 55, 61, 56, 70, 11, 63, 1, 69, 57, 36, 13, 338]], [[21], [61, 56, 4, 57, 5, 429, 7, 55, 63, 62, 68, 13, 36, 11, 69, 1]], [[1], [7, 429, 60, 61, 15, 56, 55, 57, 62, 63, 13, 36, 11, 69, 1, 205]], [[7], [55, 56, 15, 57, 102, 429, 7, 61, 62, 643, 63, 13, 36, 11, 69, 1]], [[71], [61, 56, 4, 57, 5, 429, 7, 55,

In [5]:
# to load preTrained Emb from previous model (RETAIN)
'''
from collections import OrderedDict
def load_params(model):
    params = OrderedDict()
    weights = np.load(model)
    for k,v in weights.iteritems():
        params[k] = v
    return params

param= load_params("/data/projects/cerner_dev/cl2_hosp_retain_out/h50_cl2_all_hosp_143_cl2m.out.0.npz")
emb_pretrain = param['W_emb']
'''

'\nfrom collections import OrderedDict\ndef load_params(model):\n    params = OrderedDict()\n    weights = np.load(model)\n    for k,v in weights.iteritems():\n        params[k] = v\n    return params\n\nparam= load_params("/data/projects/cerner_dev/cl2_hosp_retain_out/h50_cl2_all_hosp_143_cl2m.out.0.npz")\nemb_pretrain = param[\'W_emb\']\n'

In [6]:
class EHR_RNN(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='LSTM',bi=False , preTrainEmb=''):
        super(EHR_RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        if bi: self.bi=2 
        else: self.bi=1
              
        if len(self.preTrainEmb)>0:
            emb_t= torch.FloatTensor(np.asmatrix(self.preTrainEmb))
            self.embed= nn.Embedding.from_pretrained(emb_t)#,freeze=False)  
        else:
            self.embed= nn.Embedding(input_size, self.embed_dim,padding_idx=0)#,scale_grad_by_freq=True)
        
        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "RNN":
            cell = nn.RNN
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        else:
            raise NotImplementedError
      
        self.rnn_c = cell(self.embed_dim+1, hidden_size,num_layers=n_layers, dropout= dropout_r , bidirectional=bi , batch_first=True )
        self.out = nn.Linear(self.hidden_size*self.bi,1)
        self.sigmoid = nn.Sigmoid()
        #hl=int(self.hidden_size/4)
        #print (hl)
        #self.hid1_l= nn.Linear(3,hl)
        #self.hid2_l= nn.Linear(hl,self.hidden_size)
        #self.relu = nn.ReLU()
        #self.hid_l= nn.Linear(self.embed_dim,self.hidden_size)
        
    def EmbedPatient_MB(self, input): # x is a ehr_seq_tensor
        
        mb=[]
        mtd=[]
        lbt=[]
        seq_l=[]
        self.bsize=len(input)
        lp= len(max(input, key=lambda xmb: len(xmb[-1]))[-1])
        #print (max(input, key=lambda xmb: len(xmb[-1])),lp) #verified
        llv=0
        for x in input:
            lv= len(max(x[-1], key=lambda xmb: len(xmb[1]))[1])
            #print(max(x[-1], key=lambda xmb: len(xmb[1:])),lv) #verified  
            if llv < lv:
                llv=lv             
        #print (llv)
        for pt in input:
            sk,label,ehr_seq_l = pt
            lpx=len(ehr_seq_l)
            seq_l.append(lpx)
            label_tensor = Variable(torch.FloatTensor([[float(label)]]))
            if use_cuda:
                label_tensor = label_tensor.cuda()
            lbt.append(label_tensor)
            if use_cuda:
                    flt_typ=torch.cuda.FloatTensor
                    lnt_typ=torch.cuda.LongTensor
            else: 
                lnt_typ=torch.LongTensor
                flt_typ=torch.FloatTensor
            ml=(len(max(ehr_seq_l, key=len)))
            ehr_seq_tl=[]
            time_dim=[]
            for ehr_seq in ehr_seq_l:
                #print (ehr_seq,ehr_seq[1])#verified
                #print(n_ehr_seq)
                pd=(0, (llv -len(ehr_seq[1])))
                time_dim.append(Variable(torch.from_numpy(np.asarray(ehr_seq[0],dtype=int)).type(flt_typ)))
                result = F.pad(torch.from_numpy(np.asarray(ehr_seq[1],dtype=int)).type(lnt_typ),pd,"constant", 0)
                ehr_seq_tl.append(result)
            ehr_seq_t= Variable(torch.stack(ehr_seq_tl,0)) 
            time_dim_v= Variable(torch.stack(time_dim,0))
            lpp= lp-lpx
            zp= nn.ZeroPad2d((0,0,0,lpp))
            ehr_seq_t= zp(ehr_seq_t)
            time_dim_pv= zp(time_dim_v)
            mb.append(ehr_seq_t)
            mtd.append(time_dim_pv)
            #print('ehr_seq_t',ehr_seq_t) #verified
            
        mb_t= Variable(torch.stack(mb,0)) 
        mtd_t= Variable(torch.stack(mtd,0))
        if use_cuda:
            mb_t.cuda()
            mtd_t.cuda()
        embedded = self.embed(mb_t)
        #print(mb_t,embedded) #verified
        embedded = torch.sum(embedded, dim=2) 
        lbt_t= Variable(torch.stack(lbt,0))
        #dem_t= Variable(torch.stack(demt,0))
        #if use_cuda: dem_t.cuda()
        #dem_emb=self.embed(dem_t)
        #dem_emb = torch.sum(dem_emb, dim=1) 
        #print ('embedded',embedded.shape,embedded,'time_dim_pv',mtd_t.shape,mtd_t)
        out_emb= torch.cat((embedded,mtd_t),dim=2)
        #print ('out_emb with time',out_emb.shape,out_emb)
        return out_emb, lbt_t,seq_l #,dem_emb
    
    def init_hidden(self):
        
        h_0 = Variable(torch.rand(self.n_layers*self.bi,self.bsize, self.hidden_size))
        
        if self.cell_type == "LSTM":
            result = (h_0,h_0)
        else: 
            result = h_0
            
        if use_cuda:
            return result.cuda()
        else:
            return result
    
    def forward(self, input):
        
        x_in , lt ,x_lens = self.EmbedPatient_MB(input)
        #print (dem_t.size())
        x_inp = nn.utils.rnn.pack_padded_sequence(x_in,x_lens,batch_first=True)
        
        h_0= self.init_hidden()
        #h_0 = self.sigmoid(self.hid2_l(self.sigmoid(self.hid1_l(dem_t)))).unsqueeze(0)
        #h_0 = self.sigmoid(self.hid_l(dem_t)).unsqueeze(0)
        #print (h_0)
        output, hidden = self.rnn_c(x_inp,h_0) 
        if self.cell_type == "LSTM":
            hidden=hidden[0]
        if self.bi==2:
            output = self.sigmoid(self.out(torch.cat((hidden[-2],hidden[-1]),1)))
        else:
            output = self.sigmoid(self.out(hidden[-1]))
        return output.squeeze(), lt.squeeze()

# GRU: new data format : bestValidAuc 0.831529 has a TestAuc of 0.833067 at epoch 16    
## versus bestValidAuc 0.821507 has a TestAuc of 0.837944 at epoch 2 with default h_0
# h_0 random

In [7]:
emb_pretrain=''                       
model = EHR_RNN(input_size=18000, hidden_size=64 ,embed_dim=64, dropout_r=0, cell_type='GRU',bi=False , n_layers=1, preTrainEmb=emb_pretrain)
bmodel_pth = './best_model_GRUPreReadm.pth'
bmodel_st = './best_model_GRUPreReadm.st'
if use_cuda:
    model = model.cuda()

In [8]:
def train (tmodel,mini_batch, criterion, optimizer):  
    tmodel.train()
    model.zero_grad()
    output , label_tensor = tmodel(mini_batch)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
   
    return output, loss.item()

In [9]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)





In [10]:
def run_model_train(tmodel,dataset,batch_size,learning_rate = 0.03, l2=0.001,epsl=1e-04 ):
    
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)#, weight_decay=l2)
    #optimizer = optim.Adadelta(model.parameters(), lr=learning_rate, weight_decay=l2)
    #optimizer = optim.ASGD(model.parameters(), lr=learning_rate, weight_decay=l2 )
    #optimizer = optim.SparseAdam (model.parameters(),lr=learning_rate) #'''lr=learning_rate,''' 
    optimizer = optim.Adagrad (tmodel.parameters(),lr=learning_rate, weight_decay=l2) #'''lr=learning_rate,''' 
    #optimizer = optim.Adamax(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=l2 ,eps=epsl) ### Beta defaults (0.9, 0.999)
    #optimizer = optim.RMSprop (model.parameters(),lr=learning_rate, weight_decay=l2 ,eps=epsl)
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=learning_rate)
    dataset.sort(key=lambda pt:len(pt[-1]),reverse=True) 
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    #print('number of Batches',n_batches)
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(tmodel,batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
            # Print iter number, loss, name and guess
            #if iter % print_every == 0:
               #print('%d %d%% (%s) %.4f ' % ( iter, iter/ n_batches * 100, timeSince(start), loss))

            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses


In [11]:
def calculate_auc(test_model, dataset, batch_size=200):
    test_model.eval()
    dataset.sort(key=lambda pt:len(pt[-1]),reverse=True) 
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []
    
    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = test_model(batch)
            y_hat.extend(output.cpu().data.view(-1).numpy())
            labelVec.extend(label_t.cpu().data.view(-1).numpy())
    auc = roc_auc_score(labelVec, y_hat)
    
    return auc

In [12]:
epochs=100
batch_size=100
current_loss_l=[]
all_losses_l=[]
train_auc_allep =[]
valid_auc_allep =[]
test_auc_allep=[]
#tests_auc_allep=[]
#testl_auc_allep=[]
bestValidAuc = 0.0
bestTestAuc = 0.0
bestValidEpoch = 0

for ep in range(epochs):
    
    #print (model.embed.weight.data[13] )
    start = time.time()
    current_loss_la,all_losses_la = run_model_train(model,train_sl,batch_size)
    train_time = timeSince(start)
    eval_start = time.time()
    train_auc = calculate_auc(model,train_sl,batch_size)
    test_auc = calculate_auc(model,test_sl,batch_size)
    #test_sh_auc = calculate_auc(model,test_sh_143,batch_size)
    #test_l_auc = calculate_auc(model,test_l_143,batch_size)
    valid_auc = calculate_auc(model,valid_sl,batch_size)
    eval_time = timeSince(eval_start)
    all_losses_l.append (all_losses_la)
    avg_loss = np.mean(all_losses_la)
    train_auc_allep.append(train_auc)
    valid_auc_allep.append(valid_auc)
    test_auc_allep.append(test_auc)
    #testl_auc_allep.append(test_l_auc)
    #tests_auc_allep.append(test_sh_auc)
    current_loss_l.append(current_loss_la)
    print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc," Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
    #print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc,"less than 5 visits history :",test_sh_auc," for more :",test_l_auc ," Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
    
    if valid_auc > bestValidAuc: 
        bestValidAuc = valid_auc
        bestValidEpoch = ep
        bestTestAuc = test_auc
        best_model = model
        torch.save(best_model, bmodel_pth)
        torch.save(best_model.state_dict(), bmodel_st)
    if ep - bestValidEpoch >12: break
            
print ('bestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc, bestValidEpoch))
torch.save(best_model, bmodel_pth)



Epoch  0  Train_auc : 0.7235692920655318  , Valid_auc :  0.6831921196010972  ,& Test_auc :  0.6970752327651648  Avg Loss:  0.6578739070892334 Train Time (1m 42s) Eval Time (2m 22s)


/home/lgindybekhet/.local/lib/python3.5/site-packages/torch/serialization.py:193: UserWarning:

Couldn't retrieve source code for container of type EHR_RNN. It won't be checked for correctness upon loading.



Epoch  1  Train_auc : 0.7419711697295588  , Valid_auc :  0.7125717175592488  ,& Test_auc :  0.718419382701365  Avg Loss:  0.6316814613342285 Train Time (1m 40s) Eval Time (2m 0s)
Epoch  2  Train_auc : 0.7597594597662243  , Valid_auc :  0.7246880301743144  ,& Test_auc :  0.7335426574741732  Avg Loss:  0.6272083878517151 Train Time (0m 51s) Eval Time (1m 13s)
Epoch  3  Train_auc : 0.7672895735327203  , Valid_auc :  0.7205475322682305  ,& Test_auc :  0.7374369707865784  Avg Loss:  0.6279473633766175 Train Time (0m 52s) Eval Time (1m 14s)
Epoch  4  Train_auc : 0.7716074128611068  , Valid_auc :  0.7196930251793094  ,& Test_auc :  0.7382253613923999  Avg Loss:  0.6171997375488282 Train Time (0m 52s) Eval Time (1m 12s)
Epoch  5  Train_auc : 0.7660005416728316  , Valid_auc :  0.7210644716879131  ,& Test_auc :  0.735150887126254  Avg Loss:  0.6179316773414613 Train Time (0m 53s) Eval Time (1m 12s)
Epoch  6  Train_auc : 0.7771484244363867  , Valid_auc :  0.7248026287926538  ,& Test_auc :  0.7464

In [13]:
import plotly.plotly as py 
import plotly.graph_objs as go
py.sign_in('LailaRasmy','mzNHzVvwYjcZwBDZx3B7')

train_auc_fg= go.Scatter(x= np.arange(epochs), y=train_auc_allep, name='train')
test_auc_fg= go.Scatter(x= np.arange(epochs), y=test_auc_allep, name='test')
#tests_auc_fg= go.Scatter(x= np.arange(epochs), y=tests_auc_allep, name='test<5')
#testl_auc_fg= go.Scatter(x= np.arange(epochs), y=testl_auc_allep, name='test5+')

valid_auc_fg= go.Scatter(x= np.arange(epochs), y=valid_auc_allep, name='valid')
valid_max = max(valid_auc_allep)
test_max = max(test_auc_allep)
#data = [train_auc_fg,test_auc_fg,valid_auc_fg,tests_auc_fg,testl_auc_fg]#,valid_auc_allep,test_auc_allep] 
data = [train_auc_fg,test_auc_fg,valid_auc_fg]
layout = go.Layout(xaxis=dict(dtick=1))
layout.update(dict(annotations=[go.Annotation(text="Max Valid", x=valid_auc_allep.index(valid_max), y=valid_max)]))
#layout.update(dict(annotations=[go.Annotation(text="Max Test", x=test_auc_allep.index(test_max), y=test_max)]))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='BiRNN_Auc')
#url = py.plot(data, filename='some-data')  # gen. online plot
#py.image.save_as(data, 'some-data.png') 